# Police Plotter
Downloads all police press logs from franklinma.gov, parses the log files, and stores them in a database. Policed incidents can then be analyzed. Original purpose was for predicting speed traps based on historical data. All data has been downloaded and update as of 5-28-2020. Go to [Map data](#map_data) for plotting. 

## Download reports

In [ ]:
from collector import *

download()

## Parse reports

In [ ]:
from pdfparser import *

parse_start()

## Geocode addreses in reports

In [ ]:
from geocode import *

# Uploads addresses to geocode.io for geocoding. 
create_upload_csv("addr_upload.csv")

In [ ]:
# Parses addresses and adds them to the database from geocode.io.
# If geocoded address is inaccurate, then it isn't added and prints warning message. 
# Must manually edit the input csv into the function to get rid of inaccurate results. 
parse_geocodio("addr_download.csv")

## Map data <a id='map_data'></a>
Note: requires a valid Google Maps API Key

In [2]:
#Get Google Maps API key
import os
gmaps_key = os.environ.get("GMAPS_KEY")

if gmaps_key is None:
    print("Google Maps API Key not set in Environment Variables")

In [5]:
import constants
import sqlite3
import gmaps
import gmaps.datasets

conn = sqlite3.connect(constants.DB_DIR)
c = conn.cursor()

gmaps.configure(api_key=gmaps_key)
use_street_centers = False

cmd = "SELECT * FROM incidents WHERE type='MOTOR VEHICLE, VIOLATION'"
records = c.execute(cmd).fetchall()

points = {}
for rec in records:
    addr = rec[2]
    coords = c.execute("SELECT * FROM coords WHERE address=?", (addr,)).fetchone()
    if coords is None or (not use_street_centers and coords[3] == 1):
        continue
    coord_pair = (float(coords[1]), float(coords[2]))
    if coord_pair not in points:
        points[coord_pair] = 1.0
    else:
        points[coord_pair] += 1

coord_plot = []
weights = []
for (lat, lon), v in points.items():
    coord_plot.append((lat, lon))
    weights.append(v)



In [6]:
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(coord_plot, 
                                  point_radius=20, 
                                  weights=weights,
                                  opacity=0.8,
                                  dissipating=True,
                                  max_intensity=150
                                 ))

fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# scratch pad
dict = {(123, 234): 0}
dict[(123,234)] += 1
print(dict[(123,234)])
